In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
path.seurat = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/obj.seurat.rds"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/pairs.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR/BMMC5.merge.240528/"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import seurat object

In [5]:
obj.seurat = readRDS(path.seurat)

In [6]:
obj.seurat = subset(obj.seurat,
                    cells = colnames(obj.seurat)[!is.na(obj.seurat$cell_type.merged)])

Import candidate E-G pairs

In [7]:
pairs.E2G = readRDS(path.pairs.E2G)

Create an ArchRProject

In [8]:
str(Fragments(obj.seurat))

List of 13
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "86350e844e0a3a1991e9c650ccac7f43" "555000121b665f095f13712669597ac8"
  .. ..@ cells: Named chr [1:5315] "TAGTTGTCACCCTCAC-1" "CTATGGCCATAACGGG-1" "CCGCACACAGGTTAAA-1" "TCATTTGGTAATGGAA-1" ...
  .. .. ..- attr(*, "names")= chr [1:5315] "s1d1_TAGTTGTCACCCTCAC-1" "s1d1_CTATGGCCATAACGGG-1" "s1d1_CCGCACACAGGTTAAA-1" "s1d1_TCATTTGGTAATGGAA-1" ...
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "3ee058ae058031e3f233c68b0db9e6b9" "1039e761a9da92608ebabb952d11b3c8"
  .. ..@ cells: Named chr [1:5246] "GATGCAGCAGCAACAG-1" "TGAGTGAAGAGGAAGG-1" "CCGCTAAAGGACCGCT-1" "CTAACC

In [9]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(Fragments(obj.seurat)[[1]]@path,
                 Fragments(obj.seurat)[[2]]@path,
                 Fragments(obj.seurat)[[3]]@path,
                 Fragments(obj.seurat)[[4]]@path,
                 Fragments(obj.seurat)[[5]]@path,
                 Fragments(obj.seurat)[[6]]@path,
                 Fragments(obj.seurat)[[7]]@path,
                 Fragments(obj.seurat)[[8]]@path,
                 Fragments(obj.seurat)[[9]]@path,
                 Fragments(obj.seurat)[[10]]@path,
                 Fragments(obj.seurat)[[11]]@path,
                 Fragments(obj.seurat)[[12]]@path,
                 Fragments(obj.seurat)[[13]]@path),
  sampleNames = c("s1d1",
                  "s1d2",
                  "s1d3",
                  "s2d1",
                  "s2d4",
                  "s2d5",
                  "s3d10",
                  "s3d3",
                  "s3d6",
                  "s3d7",
                  "s4d1",
                  "s4d8",
                  "s4d9"),
  validBarcodes = list(s1d1 = unname(Fragments(obj.seurat)[[1]]@cells),
                       s1d2 = unname(Fragments(obj.seurat)[[2]]@cells),
                       s1d3 = unname(Fragments(obj.seurat)[[3]]@cells),
                       s2d1 = unname(Fragments(obj.seurat)[[4]]@cells),
                       s2d4 = unname(Fragments(obj.seurat)[[5]]@cells),
                       s2d5 = unname(Fragments(obj.seurat)[[6]]@cells),
                       s3d10 = unname(Fragments(obj.seurat)[[7]]@cells),
                       s3d3 = unname(Fragments(obj.seurat)[[8]]@cells),
                       s3d6 = unname(Fragments(obj.seurat)[[9]]@cells),
                       s3d7 = unname(Fragments(obj.seurat)[[10]]@cells),
                       s4d1 = unname(Fragments(obj.seurat)[[11]]@cells),
                       s4d8 = unname(Fragments(obj.seurat)[[12]]@cells),
                       s4d9 = unname(Fragments(obj.seurat)[[13]]@cells)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-1eabfc4af6488f-Date-2024-05-29_Time-16-11-54.440132.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2024-05-29 16:11:56.850904 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-1eabfc4af6488f-Date-2024-05-29_Time-16-11-54.440132.log



In [10]:
ArrowFiles

[1] "s3d10.arrow" "s3d3.arrow"  "s3d7.arrow"  "s4d1.arrow"  "s1d1.arrow" 
 [6] "s4d8.arrow"  "s2d4.arrow"  "s2d1.arrow"  "s2d5.arrow"  "s1d2.arrow" 
[11] "s1d3.arrow"  "s4d9.arrow"  "s3d6.arrow"

In [11]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

Warning message in dir.create(paste(dir.output, "ArchR", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR/BMMC5.merge.240528//ArchR' already exists”


In [12]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...



Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 


Getting Cell Metadata...



Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__           

In [13]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR/BMMC5.merge.240528/ArchR 
samples(13): s3d10 s3d3 ... s4d9 s3d6
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 55479
medianTSS(1): 16.031
medianFrags(1): 9711

In [14]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-1eabfcba8f4fc-Date-2024-05-29_Time-16-51-55.369859.log
If there is an issue, please report to github with logFile!

2024-05-29 16:52:01.206042 : Computing Total Across All Features, 0.059 mins elapsed.

2024-05-29 16:52:40.185644 : Computing Top Features, 0.708 mins elapsed.

###########
2024-05-29 16:52:43.429806 : Running LSI (1 of 2) on Top Features, 0.762 mins elapsed.
###########

2024-05-29 16:52:43.48296 : Sampling Cells (N = 10007) for Estimated LSI, 0.763 mins elapsed.

2024-05-29 16:52:43.495954 : Creating Sampled Partial Matrix, 0.763 mins elapsed.

2024-05-29 16:52:58.678342 : Computing Estimated LSI (projectAll = FALSE), 1.016 mins elapsed.

2024-05-29 16:54:02.726497 : Identifying Clusters, 2.084 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2024-05-29 16:54:18.182232 : Identified 6 Clusters, 2.341 mins elapsed.

2024-05-29 16:54:18.464996 : Saving LSI Iteration, 2.3


************************************************************
2024-05-29 16:54:34.471288 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-1eabfcba8f4fc-Date-2024-05-29_Time-16-51-55.369859.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2024-05-29 16:54:34.474272 : Creating Cluster Matrix on the total Group Features, 2.613 mins elapsed.

2024-05-29 16:55:07.897744 : Computing Variable Features, 3.17 mins elapsed.

###########
2024-05-29 16:55:08.106735 : Running LSI (2 of 2) on Variable Features, 3.174 mins elapsed.
###########

2024-05-29 16:55:08.156132 : Creating Partial Matrix, 3.174 mins elapsed.

2024-05-29 16:55:56.723484 : Computing LSI, 3.984 mins elapsed.

2024-05-29 16:57:18.068413 : Finished Running IterativeLSI, 5.34 mins elapsed.



In [15]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [16]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-1eabfc5ba452b3-Date-2024-05-29_Time-16-57-21.90611.log
If there is an issue, please report to github with logFile!

2024-05-29 16:57:22.287047 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addPeakMatrix-1eabfc5ba452b3-Date-2024-05-29_Time-16-57-21.90611.log



In [25]:
paths.h5 = c("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor2/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor4/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor5/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor10/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor6/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor7/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor8/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor9/outs/raw_feature_bc_matrix.h5")
names.sample = c("s1d1",
                 "s1d2",
                 "s1d3",
                 "s2d1",
                 "s2d4",
                 "s2d5",
                 "s3d10",
                 "s3d3",
                 "s3d6",
                 "s3d7",
                 "s4d1",
                 "s4d8",
                 "s4d9")
list.seRNA = lapply(1:length(names.sample),function(x){
    import10xFeatureMatrix(paths.h5[x],
                           names.sample[x])
})

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.


In [26]:
id.intersect = rowData(list.seRNA[[1]])[,"id"]
length(id.intersect)
for(n.tmp in 2:13){
    id.intersect = intersect(id.intersect,rowData(list.seRNA[[n.tmp]])[,"id"])
}
length(id.intersect)

[1] 61218

[1] 61165

In [27]:
seRNA = list.seRNA[[1]][rowData(list.seRNA[[1]])[,"id"] %in% id.intersect]
for(n.tmp in 2:13){
    seRNA = cbind(seRNA,list.seRNA[[n.tmp]][rowData(list.seRNA[[n.tmp]])[,"id"] %in% id.intersect])
}
seRNA

class: RangedSummarizedExperiment 
dim: 61165 9222766 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(9222766): s1d1#AAACAGCCAAACAACA-1 s1d1#AAACAGCCAAACATAG-1 ...
  s4d9#TTTGTTGGTTTGTGGA-1 s4d9#TTTGTTGGTTTGTTGC-1
colData names(0):

In [28]:
seRNA = seRNA[,rownames(obj.ArchR@cellColData)]

In [29]:
seRNA

class: RangedSummarizedExperiment 
dim: 61165 55479 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(55479): s3d10#GTAGCTGTCAGCAAAG-1 s3d10#AACTAGTGTTACCTGT-1 ...
  s3d6#TCACCGGCAGCCTTGG-1 s3d6#GAACCTGTCGTTACAA-1
colData names(0):

In [30]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d7ae6278add54-Date-2024-05-31_Time-14-44-24.127413.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2024-05-31 14:44:25.627617 : 

Overlap Per Sample w/ scATAC : s1d1=5255,s1d2=5205,s1d3=3342,s2d1=3446,s2d4=4835,s2d5=3660,s3d10=5404,s3d3=3500,s3d6=1421,s3d7=1319,s4d1=6035,s4d8=8730,s4d9=3327

2024-05-31 14:44:25.716954 : 

2024-05-31 14:44:34.436092 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d7ae6278add54-Date-2024-05-31_Time-14-44-24.127413.log



Run ArchR prediction

In [31]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-1d7ae65ac0b9fd-Date-2024-05-31_Time-14-48-29.075727.log
If there is an issue, please report to github with logFile!

2024-05-31 14:48:31.263563 : Getting Available Matrices, 0.036 mins elapsed.

2024-05-31 14:48:45.024492 : Filtered Low Prediction Score Cells (0 of 55479, 0), 0.105 mins elapsed.

2024-05-31 14:48:51.087551 : Computing KNN, 0.206 mins elapsed.

2024-05-31 14:48:51.552443 : Identifying Non-Overlapping KNN pairs, 0.214 mins elapsed.

2024-05-31 14:48:54.008135 : Identified 500 Groupings!, 0.255 mins elapsed.

2024-05-31 14:48:54.393522 : Getting Group RNA Matrix, 0.261 mins elapsed.

2024-05-31 14:49:44.170898 : Getting Group ATAC Matrix, 1.091 mins elapsed.

2024-05-31 14:51:11.088246 : Normalizing Group Matrices, 2.539 mins elapsed.

2024-05-31 14:51:22.816461 : Finding Peak Gene Pairings, 2.735 mins elapsed.

2024-05-31 14:51:28.099774 : Computing Correlations, 2.823 mins elapsed.

2024-05-31 14:52:38.297196 : Comple

In [32]:
execution_time

Time difference of 4.192262 mins

In [33]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [34]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [35]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [36]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 17988882 rows and 7 columns
                                  idxATAC    idxRNA Correlation       FDR
                                <integer> <integer>   <numeric> <numeric>
chr1-10010-10512_DDX11L1                1         1         NaN       NaN
chr1-115495-115916_DDX11L1              2         1         NaN       NaN
chr1-180477-181904_DDX11L1              3         1         NaN       NaN
chr1-191243-191960_DDX11L1              4         1         NaN       NaN
chr1-267886-268118_DDX11L1              5         1         NaN       NaN
...                                   ...       ...         ...       ...
chrX-156008487-156008683_WASH6P    303478     60591  -0.0662813  0.344825
chrX-156008873-156009316_WASH6P    303479     60591   0.0114688  0.897834
chrX-156019835-156020122_WASH6P    303480     60591  -0.0337040  0.697112
chrX-156029840-156030373_WASH6P    303481     60591  -0.0129684  0.884018
chrX-156030498-156030725_WASH6P    303482     60591  -0.0365152  0.66

In [37]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [38]:
pairs.E2G.res

GRanges object with 5444382 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100015450-100016026      * |         AGL
        [2]     chr1 100015450-100016026      * |      CDC14A
        [3]     chr1 100015450-100016026      * |         DBT
        [4]     chr1 100015450-100016026      * |       EXTL2
        [5]     chr1 100015450-100016026      * |     MFSD14A
        ...      ...                 ...    ... .         ...
  [5444378]     chrX     9995615-9996434      * |        WWC3
  [5444379]     chrX     9997010-9998101      * |       CLCN4
  [5444380]     chrX     9997010-9998101      * |     SHROOM2
  [5444381]     chrX     9997010-9998101      * |       TBL1X
  [5444382]     chrX     9997010-9998101      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

Save results

In [39]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 5444382 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100015450-100016026      * |         AGL
        [2]     chr1 100015450-100016026      * |      CDC14A
        [3]     chr1 100015450-100016026      * |         DBT
        [4]     chr1 100015450-100016026      * |       EXTL2
        [5]     chr1 100015450-100016026      * |     MFSD14A
        ...      ...                 ...    ... .         ...
  [5444378]     chrX     9995615-9996434      * |        WWC3
  [5444379]     chrX     9997010-9998101      * |       CLCN4
  [5444380]     chrX     9997010-9998101      * |     SHROOM2
  [5444381]     chrX     9997010-9998101      * |       TBL1X
  [5444382]     chrX     9997010-9998101      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100015450,100016026,AGL,K562,-0.0497185644,5.247321e-01,0.2082232,0.9247413
chr1,100015450,100016026,CDC14A,K562,-0.0293855869,7.390156e-01,0.2082232,0.9897840
chr1,100015450,100016026,DBT,K562,0.0702857420,3.053720e-01,0.2082232,0.8568929
chr1,100015450,100016026,EXTL2,K562,0.1265928406,2.352107e-02,0.2082232,0.8007955
chr1,100015450,100016026,MFSD14A,K562,-0.0257199604,7.724835e-01,0.2082232,0.7414798
chr1,100015450,100016026,RTCA,K562,-0.0004325087,9.964575e-01,0.2082232,0.8824248
chr1,100015450,100016026,RTCA-AS1,K562,0.0245631777,7.827345e-01,0.2082232,0.7876252
chr1,100015450,100016026,SASS6,K562,0.0496383256,5.256275e-01,0.2082232,0.9391824
chr1,100015450,100016026,SLC30A7,K562,0.0436248589,5.925560e-01,0.2082232,0.9269198


In [40]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] uwot_0.1.16            